# Practical Optimization for Stats Nerds

[Ryan J. O'Neil](mailto:ryanjoneil@gmail.com)  
Data Science DC  
March 20, 2017

## Take-aways, in case you want to skip the talk
  
  
* Many statistical techniques are based on some sort of optimization.
  


* Optimization has many other uses, such as solving decision models.
  


* Learning to structure problems you already know for optimization solvers is a great way to understand them!


## Least Squares

We observe noisy data from an unknown function. We want to infer that function.

But let's assume that, deep down, we actually know the function. That way we can generate noisy data and see if our techniques work right.

#### Function:
$$y = 3x^2 - 2x + 10 + \epsilon$$

#### Noise:
$$\epsilon \sim N\left(0, 25\right)$$

In [1]:
# Generate some random data.
import numpy as np
import random

x = []
y = []
for _ in range(500):
    xi = random.uniform(-10, 10)
    eps = random.normalvariate(0, 25)
    yi = 3*xi**2 - 2*xi + 10 + eps
    
    x.append(xi)
    y.append(yi)
    
    
x = np.array(x)
y = np.array(y)

In [2]:
from bokeh.charts import Scatter, output_notebook, show
output_notebook()

scatter = Scatter({'x': x, 'y': y}, width=750, height=400)
show(scatter)

Loading BokehJS ...

### Least Squares the way _you_ do it...

...assuming you use `scikit-learn` like every other sane Python programmer.

We looked at a chart of our data and decided to describe it with:

* A quadratic term  
* A linear term
* An offset

In [3]:
from sklearn.linear_model import LinearRegression

# Note: A is our feature matrix.
#       We intentionally add a "1" for the offset, instead of letting 
#       sklearn do that for us. This will make sense soon.
X = np.array([[xi**2, xi, 1] for xi in x])

lin = LinearRegression(fit_intercept=False)
lin.fit(X, y)

print(lin.coef_)

[ 3.04754276 -2.27555522  7.64319348]


/Users/roneil/src/github.com/ryanjoneil/talks/2017/03/data-science-dc/lib/python3.6/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [4]:
# How'd we do?
y_hat = lin.predict(X)
scatter = Scatter({'x': x, 'y': y_hat}, width=750, height=400)
show(scatter)

### Least Squares the way your _grandparents_ did it...

...with chalk and a slab of slate.

Construct a function to calculate the sum of squared residuals...

$$
\begin{align}
    \text{min}\ f(\beta) & = \frac{1}{2} ||y - X \beta||^2 \\
                         & \\
                         & = \frac{1}{2} (y - X \beta)'(y - X \beta) \\
                         & \\
                         & = \frac{1}{2} y'y - y'X\beta + \frac{1}{2} \beta'X'X\beta \\
                         & \\
                         & = \frac{1}{2} y'y - y'X\beta + \frac{1}{2} \beta'X'X\beta \\
                         & \\
\end{align}
$$

#### First Order Necessary Conditions

...and take its derivative to find a closed-form solution.

$$\nabla f(\beta) = X'X\beta - y'X\beta = 0$$  
$$\beta = (X'X)^{-1}X'y$$

In [5]:
from numpy.linalg import inv

# beta = (X'X)^-1 * X * y
Xt = X.transpose()
pseudo_inv = inv(np.matmul(Xt, X))
beta = np.matmul(np.matmul(pseudo_inv, Xt), y)
print(beta)

[ 3.04754276 -2.27555522  7.64319348]


In [6]:
# How'd grandpa and grandma do?
y_hat = [beta[0]*xi**2 + beta[1]*xi + beta[2] for xi in x]
scatter = Scatter({'x': x, 'y': y_hat}, width=750, height=400)
show(scatter)

### Least Squares the way your _crazy uncle Eddie_ does it...

...'cause he used to work at NASA and code in Forth.

[`cvxopt`](http://cvxopt.org/) provides a [`qp`](http://cvxopt.org/userguide/coneprog.html#quadratic-programming) method that can solve anything of this form.

$$
\begin{align}
    \text{min}  \ \ \ & \ \frac{1}{2} \beta'P\beta + q'\beta \\
                      & \\
    \text{s.t.} \ \ \ & \ G\beta \preceq h \\
                      & \\
                      & \ A\beta = b
\end{align}
$$


So we need to convert from 

$$\frac{1}{2} \beta'X'X\beta - y'X\beta + \frac{1}{2} y'y $$

to another form

$$\frac{1}{2} \beta'P\beta + q'\beta$$

which is simply

$$P = X'X, q = -y$$

In [7]:
import cvxopt as cvx

P = cvx.matrix(np.matmul(Xt, X))
q = cvx.matrix(-1 * np.matmul(y.transpose(), X))
solution = cvx.solvers.qp(P, q)
beta = solution['x'] # unrelated to our x
print(beta)

[ 3.05e+00]
[-2.28e+00]
[ 7.64e+00]



In [8]:
# How'd Crazy Uncle Eddie do?
y_hat = [beta[0]*xi**2 + beta[1]*xi + beta[2] for xi in x]
scatter = Scatter({'x': x, 'y': y_hat}, width=750, height=400)
show(scatter)

#### So what's different about Crazy Uncle Eddie?

Well, besides the obvious.

![](images/crazy-uncle-eddie.jpg)

While all three techniques produced the _same result_, Crazy Uncle Eddie's is interesting because it is more general than the others.

Crazy Eddie can solve any quadratic optimization problem, of which Least Squares is _just one instance_.

If we change the structure of the problem slightly:

* We probably can't solve it with `scikit-learn`
* Grams and Gramps have to go back to their chalkboard
* Crazy Eddie can update the inputs to his problem and reoptimize

## Example: Portfolio Optimization

